[Erin Swenson-Healey: A Beginner's Guide to Exceptions in Haskell](https://www.youtube.com/watch?v=PWS0Whf6-wc?t=1290)

In [61]:
{-# LANGUAGE BangPatterns #-}

In [68]:
:t readFile
readFile "bogus"

readFile :: FilePath -> IO String

bogus: openFile: does not exist (No such file or directory)

In [11]:
f x = Just x
g x = Just x
aux = do
  m1 <- f "foo"
  m2 <- g "bar"    -- BAD enforces order, first m1 then m2
  return (m1 ++ m2)
:t aux  
aux 

aux :: Maybe [Char]

Just "foobar"

In [20]:
aux = (++) <$> f "foo" <*> g "bar" -- not so readable as do notation
:t aux
aux

aux :: Maybe [Char]

Just "foobar"

In [25]:
:t error
:t undefined

error :: forall a. [Char] -> a

undefined :: forall a. a

In [37]:
import Control.Concurrent
import Control.Exception --(ErrorCall)
:t ErrorCall --constructor (:t Type  ...->ErrorCall)

main = do
  tid <-  forkIO sleepyFn
  threadDelay 1
  throwTo  tid (ErrorCall "Abort")
  threadDelay 3
  
sleepyFn = do
  putStrLn "sleeping..." 
  threadDelay 4
  putStrLn "woke up!"
main  

ErrorCall :: String -> ErrorCall

sleeping...
woke up!

In [46]:
:t userError

userError :: String -> IOError

In [48]:
:t ioError

ioError :: forall a. IOError -> IO a

In [49]:
:t catch

catch :: forall e a. Exception e => IO a -> (e -> IO a) -> IO a

In [43]:
:t throw

throw :: forall a e. Exception e => e -> a

In [44]:
:t throwIO

throwIO :: forall a e. Exception e => e -> IO a

In [116]:
:t mask

mask :: forall b. ((forall a. IO a -> IO a) -> IO b) -> IO b

In [58]:
answer = 42 + throw (ErrorCall "Katastrofa")
:t answer
answer

Line 1: Use error
Found:
throw (ErrorCall "Katastrofa")
Why not:
error "Katastrofa"

answer :: forall a. Num a => a

Katastrofa

In [69]:
answer = 42 + error "Katastrofa"
:t answer
answer

answer :: forall a. Num a => a

Katastrofa
CallStack (from HasCallStack):
  error, called at <interactive>:1:15 in interactive:Ghci776

In [96]:
fileContent fn = catch (readFile fn) $ \e -> print (e:: IOException) >> return "default file content"
:t fileContent
main = do
  bogusContent <- fileContent "bogus" 
  print $ "> " ++ bogusContent
  inputTxtContent <- fileContent "input.txt" -- this file should be present
  print $ "> " ++ inputTxtContent
  print "Finished."
main

fileContent :: FilePath -> IO String

bogus: openFile: does not exist (No such file or directory)
"> default file content"
"> This is a test."
"Finished."

#### Enforce evaluation to [WHNF (weak-head-normal-form)](https://wiki.haskell.org/Weak_head_normal_form)  in context of IO monad
this may be usefull to avoid sync exceptions during some critical path of code

An expression is in weak head normal form (WHNF), if it is either:

1. a **constructor** (eventually applied to arguments) like True, Just (square 42) or (:) 1
1. a **built-in function** applied to **too few arguments** (perhaps none) like (+) 2 or sqrt.
1. or a **lambda abstraction** \x -> expression.

Note that the arguments ***do not themselves have to be fully evaluated*** for an expression to be in weak head normal form; thus, while (square 42) can be reduced to (42 * 42), which can itself be reduced to a normal form of 1764, Just (square 42) is WHNF without further evaluation. Similarly, (+) (2 * 3 * 4) is WHNF, even though +(2 * 3 * 4) could be reduced to the normal form 24.



In [103]:
:t evaluate
:t evaluate $ Just $ error "Dupa!"


evaluate :: forall a. a -> IO a

evaluate $ Just $ error "Dupa!" :: forall a. IO (Maybe a)

In [105]:
:t catches

catches :: forall a. IO a -> [Handler a] -> IO a

In [106]:
:t try

try :: forall e a. Exception e => IO a -> IO (Either e a)

In [110]:
:t finally -- IO b what to run always (both cases: exc thrown or not)

finally :: forall b a. IO a -> IO b -> IO a

#### [Timeout](https://www.stackage.org/haddock/lts-9.4/base/System-Timeout.html#v:timeout)
This combinator relies on asynchronous exceptions internally. The technique works very well for computations executing **inside of the Haskell runtime system**, but it ***doesn't work at all for non-Haskell code***

In [113]:
import System.Timeout
:t timeout

timeout :: forall a. Int -> IO a -> IO (Maybe a)

```
main = do
  v <- timeout 5 $ worker action
  putStrLn (show v)

worker :: (Conn -> IO a) -> IO a
worker workToDo = mask $ \unmask -> do
  conn <- leaseConn pool
  result <- unmask (workToDo conn) `catch` \e -> 
    release comm >> throwIO e
  release conn
  return result
```

https://www.youtube.com/watch?v=PWS0Whf6-wc&t=40m1s
only in `unmask` some exception can be caught !!! Other parts block/prevent delivery of exceptions

In [136]:
acquire :: IO String
--acquire = (++) <$> getContents <*> (return "_DUPA_")
--<stdin>: hGetContents: illegal operation (handle is closed)
acquire = (++) <$> (return "_BLADA_") <*> (return "_DUPA_")

release :: IO () 
release = print "Releasing..."

safeWork = mask $ \restore -> do
  x <- acquire
  restore (do_something_with x) `onException` release
  release 

do_something_with x = print x

safeWork

"_BLADA__DUPA_"
"Releasing..."

In [137]:
:t onException

onException :: forall b a. IO a -> IO b -> IO a